# Tweet Disaster ML Competition

In [1]:
#import train data
import pandas as pd
import numpy as np
df = pd.read_csv('train_tweet.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [2]:
df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [3]:
df.shape

(7613, 5)

Feature Engineer Process  
1. Use CountVectorizer to convert text to numeric vector


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

def feature_eng(x):
    X = x.text
    y = x.target
    cv = CountVectorizer()
    X = cv.fit_transform(X)
    return X, y, cv

X, y, cv = feature_eng(df)


In [5]:
X.shape

(7613, 21637)

In [6]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, test_size=0.25, random_state=42)

In [7]:
#build classification models, start with SVM
from sklearn.metrics import classification_report
from sklearn.svm import SVC

svm = SVC()
svm.fit(Xtrain, ytrain)
pred = svm.predict(Xtest)

print(classification_report(ytest,pred))

              precision    recall  f1-score   support

           0       0.78      0.93      0.85      1091
           1       0.87      0.65      0.74       813

    accuracy                           0.81      1904
   macro avg       0.83      0.79      0.80      1904
weighted avg       0.82      0.81      0.80      1904



In [8]:
#use random forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(Xtrain, ytrain)
pred = rf.predict(Xtest)

print(classification_report(ytest,pred))

              precision    recall  f1-score   support

           0       0.76      0.94      0.84      1091
           1       0.88      0.60      0.72       813

    accuracy                           0.80      1904
   macro avg       0.82      0.77      0.78      1904
weighted avg       0.81      0.80      0.79      1904



In [9]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
mlp.fit(Xtrain,ytrain)
pred = mlp.predict(Xtest)

print(classification_report(ytest,pred))

              precision    recall  f1-score   support

           0       0.80      0.83      0.81      1091
           1       0.76      0.71      0.74       813

    accuracy                           0.78      1904
   macro avg       0.78      0.77      0.78      1904
weighted avg       0.78      0.78      0.78      1904



In [10]:
X.shape

(7613, 21637)

Attempt more feature engineering:  
1. lower case text
2. remove punctuation
3. lemmatization
4. removing extra whitespace
5. handling urls, emails and special characters

In [2]:
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
from nltk.stem import WordNetLemmatizer 
import re

nltk.download('wordnet')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove special characters (punctuation, etc.)
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove stop words
    text = ' '.join(word for word in text.split() if word not in ENGLISH_STOP_WORDS)
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    
    return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\natha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
X = df.text.apply(preprocess_text)


In [3]:
#create pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

def feature_eng(x):

    cv = CountVectorizer()
    X = cv.fit_transform(x)
    return X, cv

preprocess = Pipeline([
    ('preprocess_text', FunctionTransformer(lambda x : x.apply(preprocess_text), validate=False)),
    ('feature_eng', FunctionTransformer(lambda x: feature_eng(x), validate = False))
])
    

In [4]:
X, cv = preprocess.fit_transform(df.text)
y = df.target
X.shape

(7613, 16459)

In [6]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.2, random_state=42)

In [7]:
#build classification models, start with SVM
from sklearn.metrics import classification_report
from sklearn.svm import SVC

svm = SVC(kernel = 'rbf')
svm.fit(Xtrain, ytrain)
pred = svm.predict(Xtest)

print(classification_report(ytest,pred))

              precision    recall  f1-score   support

           0       0.78      0.91      0.84       874
           1       0.85      0.66      0.74       649

    accuracy                           0.80      1523
   macro avg       0.82      0.79      0.79      1523
weighted avg       0.81      0.80      0.80      1523



In [20]:
#use random forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(Xtrain, ytrain)
pred = rf.predict(Xtest)

print(classification_report(ytest,pred))

              precision    recall  f1-score   support

           0       0.77      0.88      0.82       874
           1       0.79      0.64      0.71       649

    accuracy                           0.77      1523
   macro avg       0.78      0.76      0.76      1523
weighted avg       0.78      0.77      0.77      1523



In [22]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
mlp.fit(Xtrain,ytrain)
pred = mlp.predict(Xtest)

print(classification_report(ytest,pred))

              precision    recall  f1-score   support

           0       0.78      0.80      0.79       874
           1       0.72      0.69      0.70       649

    accuracy                           0.75      1523
   macro avg       0.75      0.74      0.75      1523
weighted avg       0.75      0.75      0.75      1523



In [35]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm, Xtrain, ytrain, cv = 5, scoring = 'accuracy').mean()

0.7963875205254516

In [36]:
#load test data
test = pd.read_csv('test_tweet.csv')
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [37]:
samp = test['text'].apply(preprocess_text)
samp = cv.transform(samp)
samp.shape

(3263, 16459)

In [38]:
predictions = svm.predict(samp)
predictions.shape

(3263,)

In [39]:
test['target']=predictions
sub = test[['target']]
sub.index = test.id
sub

,target
id,
0,1
2,1
3,1
9,1
11,1
...,...
10861,1
10865,1
10868,1


In [40]:
sub.to_csv('sub_tweet5.csv')